In [ ]:
import os 
from dotenv import load_dotenv

load_dotenv()

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [5]:
loader = PyPDFLoader("attention.pdf")
text = loader.load()

In [6]:
docs = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents = docs.split_documents(text)

In [7]:
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)

In [8]:
db = FAISS.from_documents(documents,embeddings)
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002A72A324A90>, search_kwargs={})

In [9]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7
)

In [10]:
prompt = ChatPromptTemplate.from_template(

    """Your task is to answer the following question based on the following context provided to you

    and always try to answer based on thecontext if you dont know the answer just say I dont no
    
    <context>
    {context}
    </context>
    
    question: {input}"""
)

In [11]:
llm_chain = create_stuff_documents_chain(llm,prompt)
retriever_chain = create_retrieval_chain(retriever,llm_chain)

In [12]:
response = retriever_chain.invoke({"input" : "Who are the authors of research paper attention is all you need?"})
response["answer"]

'The authors of the research paper "Attention is All You Need" are Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin.'

In [13]:
response = retriever_chain.invoke({"input" : "What does Multi-Head Attention do?"})
response["answer"]

'Multi-Head Attention allows the model to jointly attend to information from different representation subspaces at different positions.  A single attention head, by contrast, averages this information, which is less effective.  It consists of several attention layers running in parallel.'